In [1]:
!pip install datasets transformers
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import pipeline
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup

GBERT1: Finetuning Gbert-base with twitter-dataset

In [3]:
twitter_dataset = load_dataset("Alienmaster/german_politicians_twitter_sentiment")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/326k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/81.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1428 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/357 [00:00<?, ? examples/s]

In [4]:
print(twitter_dataset)

DatasetDict({
    train: Dataset({
        features: ['ID', 'majority_sentiment', 'text'],
        num_rows: 1428
    })
    test: Dataset({
        features: ['ID', 'majority_sentiment', 'text'],
        num_rows: 357
    })
})


In [5]:
train_data = twitter_dataset["train"]
test_data = twitter_dataset["test"]

In [6]:
model_name = "deepset/gbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

In [7]:
#add new column "label" with numerical values for sentiment: 0:positive, 1:negative, 2:neutral
label_mapping = {1: 0, 2: 1, 3: 2}

train_data = train_data.map(lambda x: {"label": label_mapping[x["majority_sentiment"]]})
test_data = test_data.map(lambda x: {"label": label_mapping[x["majority_sentiment"]]})

Map:   0%|          | 0/1428 [00:00<?, ? examples/s]

Map:   0%|          | 0/357 [00:00<?, ? examples/s]

In [8]:
#delete unnecessary columns
train_data = train_data.remove_columns(["ID", "majority_sentiment"])
test_data = test_data.remove_columns(["ID", "majority_sentiment"])

In [9]:
print(train_data[0])

{'text': 'Unglaublich. Ich hatte @JanJosefLiefers noch gelobt, weil er Intensivstation besucht hatte. Das nehme ich zurück. Jetzt nutzt er den Besuch, um dem Staat Erpressung von Ungeimpften zu unterstellen. Er hat gar nichts dazu gelernt. Nori Soron ¯\\_(ツ)_/¯ @NoriSoron · 8. Okt. Widerlich. Er benutzt es um weiter zu hetzen. #allesaufdentisch #allesdichtmachen #Liefers 3.384 2.760 20.113', 'label': 1}


In [10]:
#Tokenize data 
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

In [11]:
train_tokenized = train_data.map(tokenize_function, batched=True)
test_tokenized = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/1428 [00:00<?, ? examples/s]

Map:   0%|          | 0/357 [00:00<?, ? examples/s]

In [12]:
#Convert to pytorch tensors for the model
train_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [13]:
num_labels = len(label_mapping)

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
#Defining training and evaluation parameters
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="macro")
  acc = accuracy_score(labels, preds)
  precision = precision_score(labels, preds, average="macro")
  recall = recall_score(labels, preds, average="macro")
  return {"precision": precision, "recall": recall, "acc": acc, "f1": f1}

In [17]:
batch_size = 8
logging_steps = len(train_tokenized) // batch_size
model_name = f"{model_name}-finetuned-twitter"
training_args = TrainingArguments(output_dir = model_name,
                                  num_train_epochs = 4,
                                  learning_rate = 2e-5,
                                  logging_dir=None,
                                  logging_strategy="no",
                                  report_to=None,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  evaluation_strategy="no",
                                  disable_tqdm = False,
                                  logging_steps = logging_steps,
                                  log_level="info")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset=train_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

<ipython-input-18-d5d27b2681ca>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
import wandb
wandb.init(mode="disabled")

In [24]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1,428
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 716
  Number of trainable parameters = 109,929,987


Step,Training Loss


Saving model checkpoint to deepset/gbert-base-finetuned-twitter/checkpoint-500
Configuration saved in deepset/gbert-base-finetuned-twitter/checkpoint-500/config.json
Model weights saved in deepset/gbert-base-finetuned-twitter/checkpoint-500/model.safetensors
tokenizer config file saved in deepset/gbert-base-finetuned-twitter/checkpoint-500/tokenizer_config.json
Special tokens file saved in deepset/gbert-base-finetuned-twitter/checkpoint-500/special_tokens_map.json
Saving model checkpoint to deepset/gbert-base-finetuned-twitter/checkpoint-716
Configuration saved in deepset/gbert-base-finetuned-twitter/checkpoint-716/config.json
Model weights saved in deepset/gbert-base-finetuned-twitter/checkpoint-716/model.safetensors
tokenizer config file saved in deepset/gbert-base-finetuned-twitter/checkpoint-716/tokenizer_config.json
Special tokens file saved in deepset/gbert-base-finetuned-twitter/checkpoint-716/special_tokens_map.json


Training completed. Do not forget to share your model on hug

TrainOutput(global_step=716, training_loss=0.06328173056661084, metrics={'train_runtime': 549.1014, 'train_samples_per_second': 10.402, 'train_steps_per_second': 1.304, 'total_flos': 1502903842062336.0, 'train_loss': 0.06328173056661084, 'epoch': 4.0})

In [25]:
#evaluation on testplit
results = trainer.evaluate(test_tokenized)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 357
  Batch size = 8


In [27]:
results_df = pd.DataFrame([results])

In [28]:
print(results_df)

   eval_loss  eval_precision  eval_recall  eval_acc   eval_f1  eval_runtime  \
0   1.294712        0.800513     0.806184   0.80112  0.803051        9.9909   

   eval_samples_per_second  eval_steps_per_second  epoch  
0                   35.732                  4.504    4.0  


In [29]:
#save the model
model_path = "/content/gbert_finetuned_twitter"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in /content/gbert_finetuned_twitter/config.json
Model weights saved in /content/gbert_finetuned_twitter/model.safetensors
tokenizer config file saved in /content/gbert_finetuned_twitter/tokenizer_config.json
Special tokens file saved in /content/gbert_finetuned_twitter/special_tokens_map.json


('/content/gbert_finetuned_twitter/tokenizer_config.json',
 '/content/gbert_finetuned_twitter/special_tokens_map.json',
 '/content/gbert_finetuned_twitter/vocab.txt',
 '/content/gbert_finetuned_twitter/added_tokens.json',
 '/content/gbert_finetuned_twitter/tokenizer.json')

In [30]:
import shutil
from google.colab import files

In [31]:
shutil.make_archive("/content/gbert_finetuned_twitter", 'zip', "/content/gbert_finetuned_twitter")

'/content/gbert_finetuned_twitter.zip'

In [32]:
files.download("/content/gbert_finetuned_twitter.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>